# Chapter 2: Working with Text Data

## 2.2 Tokenizing text

In [51]:
import os
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x120ef3890>)

In [52]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [53]:
import re

# split each word based on whitespace

text = "Hello, World. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [54]:
# split based on whitespace and commas and periods

result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [55]:
# strip whitespace from each item and then filter out any empty strings.

result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


In [56]:
text = "Hello, world. Is this-- a test?"

# includes other forms of punctuation
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
preprocessed = result
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [57]:
len(result)

4690

## 2.3 Converting tokens into token IDs

In [58]:
# builds a vocabulary of all the tokens

all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [59]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [60]:
# putting it all together in a tokenizer class

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [61]:
tokenizer = SimpleTokenizerV1(vocab)

In [62]:
text = """It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

In [63]:
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [64]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [65]:
tokenizer.decode(tokenizer.encode(text))

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens

In [66]:
# example of a limitation of the current tokenizer: raw-text does not include "Hello"

text = "Hello, do you like tea. is this-- a test?"
tokenizer.encode(text)

KeyError: 'Hello'

In [67]:
# add end of text and unknown words as tokens to current token list

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [68]:
len(vocab.items())

1132

In [69]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [70]:
# updates tokenizer class to include <unk> token if familiar word isn't found

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [71]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [72]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [73]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

In [74]:
# GPT-2 used BytePair encoding (BPE) as its tokenizer
# it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, 
# -enabling it to handle out-of-vocabulary words

import tiktoken

In [75]:
tokenizer = tiktoken.get_encoding("gpt2")

In [76]:
tokenizer.encode("Hello world")

[15496, 995]

In [77]:
tokenizer.decode(tokenizer.encode("Hello world"))

'Hello world'

In [78]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

tokenizer.encode(text, allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

## 2.6 Data sampling with a sliding window

In [79]:
# LLMs are trained to generate one word at a time, so we can prepare training data accordingly where the next word in a sequence is the
# target for the LLM to predict. For now, this section is simply the setup for the LLM (pre-training stage)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [80]:
enc_sample = enc_text[50:]

In [81]:
# as we want the LLM to predict the next word, the targets are the inputs+1

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [82]:
 for i in range(1, context_size+1):
     context = enc_sample[:i]
     desired = enc_sample[i]

     print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [83]:
import torch

In [84]:
torch.__version__

'2.6.0'

In [85]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [86]:
# divides data into batches of size 4, removes the last batch to avoid loss spikes

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # initialize tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [87]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [88]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [89]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [90]:
# using batch inputs here, and increasing stride to avoid overlapping between input-output
# notice the targets are always one token after the inputs

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

In [91]:
# the data is almost ready for the LLM
# now, I embed the tokens in a continuous vector representation using an embedding layer

inputs_ids = torch.tensor([2,   3,    5,   1])

In [92]:
# this would result in a 6x3 weight matrix

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(tokenizer.n_vocab, output_dim)

In [93]:
# because the embedding layer is just a more efficient implementation that is equivalent to the one-hot encoding and 
# matrix-multiplication approach it can be seen as a neural network layer that can be optimized via backpropagation

print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035],
        [-0.5880,  0.3486,  0.6603],
        [-0.2196, -0.3792,  0.7671],
        ...,
        [-0.5931,  1.0895, -0.6854],
        [ 0.7447,  0.5803, -0.4246],
        [-0.3130,  0.7558, -1.2656]], requires_grad=True)


In [94]:
# converts a token with id 3 to a three dimensional vector

embedding_layer(torch.tensor([3]))

tensor([[-1.1925,  0.6984, -1.4097]], grad_fn=<EmbeddingBackward0>)

In [95]:
# converts all tokens in input_ids into 3 dimensional vectors

embedding_layer(inputs_ids)

tensor([[-0.2196, -0.3792,  0.7671],
        [-1.1925,  0.6984, -1.4097],
        [ 0.2692, -0.0770, -1.0205],
        [-0.5880,  0.3486,  0.6603]], grad_fn=<EmbeddingBackward0>)

In [96]:
inputs_ids

tensor([2, 3, 5, 1])

 ## 2.8 Encoding word positions

In [113]:
# output_dim = 256 sets the input tokens to be encoded as 256-dimensional vectors

vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [114]:
# this results in a 8 x 4 x 256 tensor of a batch size of 8 with 4 tokens each

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [115]:
print("Token IDS:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDS:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [116]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [117]:
# example of one of the 256 vector dimensions, corresponding to the first token of the first batch

token_embeddings[0, 0]

tensor([ 2.5572e-01,  4.0256e-01, -1.2418e+00, -3.5538e-01, -1.7879e+00,
        -6.3204e-01, -2.9631e-01,  6.2682e-01, -1.3226e+00, -9.2090e-01,
        -9.6005e-02, -1.0284e-01,  1.5004e+00,  6.9171e-01,  8.5592e-01,
         5.7227e-02,  6.8837e-01,  5.6939e-01, -3.6974e-01,  2.0287e+00,
         2.2091e+00,  5.9145e-01,  2.6005e-01,  4.3499e-01,  5.7573e-02,
         1.4959e+00,  3.5999e-01,  1.3825e+00, -4.0240e-01, -1.3820e-01,
         4.0812e-02,  1.0731e+00,  1.4207e-01,  4.7325e-01,  6.2781e-01,
        -5.3984e-01,  3.3792e-01,  9.5209e-01, -2.1227e+00, -1.5468e+00,
         6.8586e-02,  1.0512e+00, -9.2698e-01,  1.0980e-01,  4.6036e-01,
        -4.1604e-01, -3.2448e-01, -1.5669e-01, -5.9218e-01, -1.1020e+00,
         4.2881e-01, -1.0630e+00,  3.3821e-01, -5.5192e-01,  1.0851e+00,
         1.6413e+00,  1.6229e-01, -3.0428e-01, -5.1214e-02,  1.8436e+00,
         2.5976e-02, -2.2142e+00, -3.5916e-01,  8.6204e-01,  7.1424e-01,
        -6.1826e-01, -1.5853e+00, -1.5914e+00, -2.0

In [122]:
# creates a second embedding layer (pos_embedding_layer)
# positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [120]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [121]:
pos_embedding_layer.weight

Parameter containing:
tensor([[-1.2826, -1.6240, -0.4873,  ...,  0.6021, -0.3921, -1.3306],
        [ 1.1205, -1.3247,  1.7293,  ..., -0.6819, -0.6186,  1.0702],
        [ 0.6131, -1.3057,  0.8526,  ...,  0.0443, -0.7291,  0.2958],
        [ 0.7650, -0.8423, -3.3804,  ..., -0.3532, -0.5664, -1.1925]],
       requires_grad=True)

In [105]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [106]:
token_embeddings.shape

torch.Size([8, 4, 256])

In [107]:
pos_embeddings.shape

torch.Size([4, 256])

In [123]:
# adds positional embedding layer's vector values onto the token embedding layer to create the input embeddings

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">